In [1]:
import torch
import torch.nn as nn
import numpy as np

from datasets.openml_datasets import OpenML_Dataset
from datasets.torch_datasets import Torch_Dataset
from utils.weakener import Weakener
from models.model import MLP
from utils.losses import PartialLoss, EMLoss, EMLoss2
from utils.trainig_testing import train_and_evaluate,warm_up

In [3]:
Data = Torch_Dataset('kmnist', batch_size=64)
Weak = Weakener(Data.num_classes)
#Weak.generate_M(model_class='pll')

In [4]:
print('Methods for Data Class','\n',[method for method in dir(Data) if (method[0] != '_') & callable(getattr(Data, method)) ],'\n'
      'Instances for Data Class','\n',[method for method in dir(Data) if (method[0] != '_') & (method not in [method for method in dir(Data) if (method[0] != '_') & callable(getattr(Data, method)) ]) ],'\n\n'
      'Methods for Weak Class','\n',[method for method in dir(Weak) if (method[0] != '_') & callable(getattr(Weak, method)) ],'\n'
      'Instances for Weak Class','\n',[method for method in dir(Weak) if (method[0] != '_') & (method not in [method for method in dir(Weak) if (method[0] != '_') & callable(getattr(Weak, method)) ]) ])

Methods for Data Class 
 ['get_data', 'get_dataloader', 'include_weak', 'transform'] 
Instances for Data Class 
 ['batch_size', 'dataset', 'num_classes', 'num_features', 'shuffle', 'test_dataset', 'test_num_samples', 'train_dataset', 'train_num_samples', 'weak_labels'] 

Methods for Weak Class 
 ['generate_M', 'generate_weak', 'generate_wl_priors', 'label_matrix', 'pll_weights', 'virtual_labels', 'virtual_matrix'] 
Instances for Weak Class 
 ['M', 'c', 'd', 'w', 'z']


In [5]:
Weak.generate_M('pll',pll_p=0.5)
Weak.M[:5,:],Weak.Z[:5,:]

(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.00217014, 0.00217014],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00217014, 0.        , 0.00217014],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00217014, 0.00217014, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.00195312, 0.00195312, 0.00195312],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.00217014, 0.        , 0.        , 0.00217014]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 1]]))

In [6]:
train_X,train_y,test_X,test_y =  Data.get_data()
print(train_X.shape)

torch.Size([60000, 784])


In [7]:
Weak.generate_weak(train_y)

(tensor([222, 452, 817,  ..., 556, 726, 866], dtype=torch.int32),
 tensor([[0., 0., 1.,  ..., 1., 1., 1.],
         [0., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 0.,  ..., 1., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 1.,  ..., 0., 0., 1.],
         [1., 1., 0.,  ..., 1., 0., 1.]], dtype=torch.float64))

In [8]:
train_y[:5,:],Weak.z[:5],Weak.w[:5,:]

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]]),
 tensor([222, 452, 817, 858, 471], dtype=torch.int32),
 tensor([[0., 0., 1., 1., 1., 0., 0., 1., 1., 1.],
         [0., 1., 1., 1., 0., 0., 1., 1., 1., 0.],
         [1., 1., 0., 0., 1., 1., 1., 1., 0., 0.],
         [1., 1., 0., 1., 1., 0., 0., 1., 0., 1.],
         [0., 1., 1., 1., 1., 0., 0., 0., 0., 1.]], dtype=torch.float64))

In [9]:
Data.include_weak(Weak.z)

In [10]:
trainloader,testloader = Data.get_dataloader()

In [11]:
print('Methods for MLP Class','\n',[method for method in dir(MLP) if (method[0] != '_') & callable(getattr(MLP, method)) ],'\n'
      'Instances for MLP Class','\n',[method for method in dir(MLP) if (method[0] != '_') & (method not in [method for method in dir(MLP) if (method[0] != '_') & callable(getattr(MLP, method))])])

Methods for MLP Class 
 ['add_module', 'apply', 'bfloat16', 'buffers', 'children', 'cpu', 'cuda', 'double', 'eval', 'extra_repr', 'float', 'forward', 'get_buffer', 'get_extra_state', 'get_parameter', 'get_submodule', 'half', 'ipu', 'load_state_dict', 'modules', 'named_buffers', 'named_children', 'named_modules', 'named_parameters', 'parameters', 'register_backward_hook', 'register_buffer', 'register_forward_hook', 'register_forward_pre_hook', 'register_full_backward_hook', 'register_load_state_dict_post_hook', 'register_module', 'register_parameter', 'requires_grad_', 'set_extra_state', 'share_memory', 'state_dict', 'to', 'to_empty', 'train', 'type', 'xpu', 'zero_grad'] 
Instances for MLP Class 
 ['T_destination', 'dump_patches']


In [24]:
mlp = MLP(Data.num_features,[Data.num_features],Data.num_classes, dropout_p=0.5, bn=True, activation =  'relu')

In [25]:
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): Linear(in_features=784, out_features=10, bias=True)
  )
  (batch_norms): ModuleList(
    (0): BatchNorm1d(784, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [27]:
M_est = torch.from_numpy(Weak.M) + torch.rand((Weak.d,Weak.c))/100
M_est /= torch.sum(M_est,dim=0,keepdim=True)
torch.sum(M_est,dim=0)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000], dtype=torch.float64)

In [28]:
torch.norm(torch.from_numpy(Weak.M) -M_est, p='fro')

tensor(0.0954, dtype=torch.float64)

In [29]:
#optim = torch.optim.Adam(mlp.parameters(),lr=1e-3)
loss = EMLoss2(Weak.d,Weak.c,M_est)
optim = torch.optim.Adam([    {'params': mlp.parameters()},    {'params': loss.M_hat, 'lr': 1e-3}], lr=1e-3)

In [30]:
mlp, results = train_and_evaluate(mlp,trainloader,testloader,optimizer=optim,loss_fn=loss,num_epochs=5,sound = 1)

Epoch 1/5: Train Loss: 0.0959, Train Acc: 0.1174, Test Acc: 0.1128
Epoch 2/5: Train Loss: 0.0706, Train Acc: 0.1115, Test Acc: 0.1002
Epoch 3/5: Train Loss: 0.0065, Train Acc: 0.1000, Test Acc: 0.1000
Epoch 4/5: Train Loss: 0.0015, Train Acc: 0.1000, Test Acc: 0.1000
Epoch 5/5: Train Loss: 0.0009, Train Acc: 0.1000, Test Acc: 0.1000


In [21]:
torch.norm(torch.from_numpy(Weak.M) -loss.M_hat, p='fro')

tensor(2.3663, dtype=torch.float64, grad_fn=<NormBackward1>)

In [ ]:
loss.M_hat==M_est

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

# Plot the first subplot (Train Loss)
ax1.plot(results['train_loss'])
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Train Loss')

# Plot the second subplot (Accuracies)
ax2.plot(results['train_acc'], label='Train Accuracy')
ax2.plot(results['test_acc'], label='Test Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Accuracies')
ax2.legend()

fig.suptitle('EM Learning Results', fontsize=18, fontweight='bold', y=1.05)
# Show the plot
plt.show()

In [ ]:
torch.mean(results['test_acc'][45:])